# Move and trim files recorded on Feb 17th, concerts

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [1]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [3]:
%reload_ext autoreload

In [4]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [6]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,LS410,5022154,230216,0,DATA-LS410-5022154-23021600.CSV,csv,567466,2023-02-16 07:10:44+00:00,2023-02-17 12:13:44+00:00,...,[LS410],5022154.0,LS410,NaN,2.0,99.0,0.0,1.0,Stationary,Prone
1,DATA,VN102,3420426,230217,0,DATA-VN102-3420426-23021700.CSV,csv,96165,2023-02-17 07:09:37+00:00,2023-02-17 12:00:07+00:00,...,[VN102],3420426.0,VN102,NaN,0.0,97.0,0.0,0.0,Stationary,Upright
2,DATA,VN205,5022060,230217,0,DATA-VN205-5022060-23021700.CSV,csv,97835,2023-02-17 07:09:43+00:00,2023-02-17 12:04:58+00:00,...,[VN205],5022060.0,VN205,NaN,55.0,99.0,0.0,0.0,Stationary,Upright
3,DATA,VN104,3420456,230217,0,DATA-VN104-3420456-23021700.CSV,csv,96190,2023-02-17 07:11:02+00:00,2023-02-17 12:00:18+00:00,...,[VN104],3420456.0,VN104,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
4,DATA,VA305,5022117,230217,0,DATA-VA305-5022117-23021700.CSV,csv,100057,2023-02-17 07:11:09+00:00,2023-02-17 12:13:39+00:00,...,[VA305],5022117.0,VA305,NaN,63.0,98.0,0.0,0.0,Stationary,Upright
5,DATA,PC707,5022256,230217,101,DATA-PC707-5022256-23021700_0.CSV,csv,92186,2023-02-17 07:11:10+00:00,2023-02-17 11:48:55+00:00,...,[PC707],5022256.0,PC707,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
6,DATA,WW506,5022166,230217,0,DATA-WW506-5022166-23021700.CSV,csv,99523,2023-02-17 07:11:12+00:00,2023-02-17 12:11:42+00:00,...,[WW506],5022166.0,WW506,NaN,92.0,97.0,0.0,0.0,Stationary,Upright
7,DATA,WW501,5022155,230217,0,DATA-WW501-5022155-23021700.CSV,csv,99151,2023-02-17 07:11:12+00:00,2023-02-17 12:11:42+00:00,...,[WW501],5022155.0,WW501,NaN,78.0,98.0,0.0,1.0,Stationary,Prone
8,DATA,LS403,5022141,230217,0,DATA-LS403-5022141-23021700.CSV,csv,100051,2023-02-17 07:11:15+00:00,2023-02-17 12:13:45+00:00,...,[LS403],5022141.0,LS403,NaN,84.0,98.0,0.0,0.0,Stationary,Upright
9,DATA,LS402,5022139,230217,0,DATA-LS402-5022139-23021700.CSV,csv,95642,2023-02-17 07:11:16+00:00,2023-02-17 11:59:16+00:00,...,[LS402],5022139.0,LS402,NaN,0.0,97.0,0.0,0.0,Stationary,Upright


In [7]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 55
[5022154 3420426 5022060 3420456 5022117 5022256 5022166 5022155 5022141
 5022139 5022240 5022076 5022073 5022002 5022077 3420415 5022006 3420427
 3420755 5022009 5022131 5022138 5022151 5023926 5022143 5022142 5022149
 5022051 5022050 5022058 5022057 5022062 5022098 5022163 5022173 5022161
 5022156 5022202 5022164 5022126 5022127 5022104 5022100 5022101 5022110
 5022204 5022203 5022193 5022198 5022184 5022179 5022244 5022245 5022241
 5022238]
['LS410' 'VN102' 'VN205' 'VN104' 'VA305' 'PC707' 'WW506' 'WW501' 'LS403'
 'LS402' 'PC702' 'VN208' 'VN207' 'VN106' 'VN209' 'VN101' 'VN107' 'VN103'
 'VN105' 'VN108' 'VA308' 'LS401' 'LS408' 'LS406' 'LS405' 'LS404' 'LS407'
 'VN202' 'VN201' 'VN204' 'VN203' 'VN206' 'VN210' 'WW504' 'WW507' 'WW503'
 'WW502' 'BR605' 'WW505' 'VA306' 'VA307' 'VA303' 'VA301' 'VA302' 'VA304'
 'BR607' 'BR606' 'BR603' 'BR604' 'BR602' 'BR601' 'PC704' 'PC705' 'PC703'
 'PC701']
[230216 230217]


# discard tiny


In [12]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230216
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


In [13]:
s_files=df_datafiles

## Define set of recordings to move

We sort files by day and time recording

In [12]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230216
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


Additionally select according to start and stop recording times.

In [9]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-17 9:34:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [10]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-17 07:00:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']> t,:] 

In [14]:
# recordings ended after a certain time
t = pd.to_datetime('2023-02-17 10:30:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [15]:
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,LS410,5022154,230216,0,DATA-LS410-5022154-23021600.CSV,csv,567466,2023-02-16 07:10:44+00:00,2023-02-17 12:13:44+00:00,...,[LS410],5022154.0,LS410,NaN,2.0,99.0,0.0,1.0,Stationary,Prone
1,DATA,VN102,3420426,230217,0,DATA-VN102-3420426-23021700.CSV,csv,96165,2023-02-17 07:09:37+00:00,2023-02-17 12:00:07+00:00,...,[VN102],3420426.0,VN102,NaN,0.0,97.0,0.0,0.0,Stationary,Upright
2,DATA,VN205,5022060,230217,0,DATA-VN205-5022060-23021700.CSV,csv,97835,2023-02-17 07:09:43+00:00,2023-02-17 12:04:58+00:00,...,[VN205],5022060.0,VN205,NaN,55.0,99.0,0.0,0.0,Stationary,Upright
3,DATA,VN104,3420456,230217,0,DATA-VN104-3420456-23021700.CSV,csv,96190,2023-02-17 07:11:02+00:00,2023-02-17 12:00:18+00:00,...,[VN104],3420456.0,VN104,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
4,DATA,VA305,5022117,230217,0,DATA-VA305-5022117-23021700.CSV,csv,100057,2023-02-17 07:11:09+00:00,2023-02-17 12:13:39+00:00,...,[VA305],5022117.0,VA305,NaN,63.0,98.0,0.0,0.0,Stationary,Upright
5,DATA,PC707,5022256,230217,101,DATA-PC707-5022256-23021700_0.CSV,csv,92186,2023-02-17 07:11:10+00:00,2023-02-17 11:48:55+00:00,...,[PC707],5022256.0,PC707,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
6,DATA,WW506,5022166,230217,0,DATA-WW506-5022166-23021700.CSV,csv,99523,2023-02-17 07:11:12+00:00,2023-02-17 12:11:42+00:00,...,[WW506],5022166.0,WW506,NaN,92.0,97.0,0.0,0.0,Stationary,Upright
7,DATA,WW501,5022155,230217,0,DATA-WW501-5022155-23021700.CSV,csv,99151,2023-02-17 07:11:12+00:00,2023-02-17 12:11:42+00:00,...,[WW501],5022155.0,WW501,NaN,78.0,98.0,0.0,1.0,Stationary,Prone
8,DATA,LS403,5022141,230217,0,DATA-LS403-5022141-23021700.CSV,csv,100051,2023-02-17 07:11:15+00:00,2023-02-17 12:13:45+00:00,...,[LS403],5022141.0,LS403,NaN,84.0,98.0,0.0,0.0,Stationary,Upright
9,DATA,LS402,5022139,230217,0,DATA-LS402-5022139-23021700.CSV,csv,95642,2023-02-17 07:11:16+00:00,2023-02-17 11:59:16+00:00,...,[LS402],5022139.0,LS402,NaN,0.0,97.0,0.0,0.0,Stationary,Upright


In [11]:
# show subset of files
s_files.iloc[0,:]

Signal                                                            DATA
DevName                                                          SPARE
ID                                                             5022299
Date                                                            230215
Session                                                              0
FileName                               DATA-SPARE-5022299-23021500.CSV
FileType                                                           csv
FileSize                                                         30091
RecStart                                     2023-02-15 06:39:12+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 08:08:57+00:00
Duration                                                        5385.0
BatteryStart                                                    4141.0
BatteryEnd                                                      4094.0
Batter

In [15]:
s_files.max()

Signal                                                            DATA
DevName                                                          WW508
ID                                                             5023926
Date                                                            230215
Session                                                            102
FileName                               DATA-WW508-5022176-23021501.CSV
FileType                                                           csv
FileSize                                                        203597
RecStart                                     2023-02-15 10:13:51+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 16:50:18+00:00
Duration                                                       37230.0
BatteryStart                                                    4170.0
BatteryEnd                                                      4158.0
Batter

In [16]:
s_files.min()

Signal                                                            DATA
DevName                                                          BR601
ID                                                             3420415
Date                                                            230215
Session                                                              0
FileName                               DATA-BR601-5022179-23021500.CSV
FileType                                                           csv
FileSize                                                           217
RecStart                                     2023-02-15 06:27:44+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 07:27:41+00:00
Duration                                                           0.0
BatteryStart                                                    4082.0
BatteryEnd                                                      3795.0
Batter

These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [16]:
projectsFolder =  'D:\\' # where qiosk files are moved to after export
print(projectsFolder) # where the qiosk files are initially saved by the Equivital Qiosk program

D:\


In [17]:
# set project folder name
projectName = '230217_Lydo_Concerts'
projectPath = projectsFolder + projectName + '\\'

In [18]:
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [19]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

# Set Interval to keep in CSV

We will cut the the CSV files to save on space and presever privacy. 
In this case, we are only taking the 

In [20]:
t1 = pd.to_datetime('2023-02-17 09:15:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]
t2 = pd.to_datetime('2023-02-17 12:30:00.00+0100')
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]

[Timestamp('2023-02-17 09:15:00+0100', tz='pytz.FixedOffset(60)'),
 Timestamp('2023-02-17 12:30:00+0100', tz='pytz.FixedOffset(60)')]

In [21]:
(t2-t1).total_seconds()

11700.0

In [54]:
# test cutting
tic = time.time()

dataLoc = s_files.iloc[0,14]
matched = qex.matched_files(dataLoc,path,seperator) # outputs locations of csv 
for fi in matched:
    if not fi.lower().endswith('.sem'):
        fp = fi.split(seperator)
        fn = fp[-1].split('.')
        fn_new = fn[0]+'-trimmed.csv'
        print(fn_new)
        V = qex.cut_by_time(fi,t1,t2)
        print(V.columns)
        V.to_csv(projectPath + 'CSV\\' + fn_new)
        print(time.time()-tic)
    if eq_file_loc.lower().endswith('sem'):
        if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
            os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
        out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
        os.rename(eq_file_loc,out_f)
        print(time.time()-tic)

DATA-VN201-5022050-23021400-trimmed.csv
Index(['SENSOR ID', 'SUBJECT ID', 'SUBJECT AGE', 'HR(BPM)', 'HRC(%)',
       'BELT OFF', 'LEAD OFF', 'MOTION', 'BODY POSITION', 'BATTERY(mV)',
       'TEMPERATURE', 'BUTTON PRESS'],
      dtype='object')
0.4191780090332031
ECG-VN201-5022050-23021400-trimmed.csv
Index(['Lead 1', 'Lead 2'], dtype='object')
47.62759590148926
FASTACC-VN201-5022050-23021400-trimmed.csv
Index(['Vert Accelerometer', 'Lat Accelerometer', 'Long Accelerometer'], dtype='object')
90.35752868652344
RESPACC-VN201-5022050-23021400-trimmed.csv
Index(['Breathing'], dtype='object')
94.61357927322388
RR-VN201-5022050-23021400-trimmed.csv
Index(['Interbeat Interval (MS)'], dtype='object')
94.8211452960968


# Full Cut Save Move Discard

In [22]:
tac = time.time()

for i,row in s_files.iterrows():
    tic = time.time()
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator)  # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                fn = fileName.split('.')
                fn_new = fn[0]+'-cut.csv'
                out_f = projectPath + 'CSV\\' + fn_new
                print(out_f)
                V = qex.cut_by_time(fi,t1,t2)
                V.to_csv(out_f) # write the trimmed csv to the external harddisk
                
                dout_f = discardPath + 'CSV\\' + fileName
                shutil.move(fi,dout_f) # move the exported csv files to the discard folder
                print(time.time()-tic)
        if fileName.lower().endswith('sem'):
            devName = row['DevName']
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)
            print(time.time()-tic)
            
    print([time.time()-tic, time.time()-tac])

dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230217_Lydo_Concerts\CSV\DATA-LS410-5022154-23021600-cut.csv
0.3553917407989502
D:\230217_Lydo_Concerts\CSV\ECG-LS410-5022154-23021600-cut.csv
51.215195655822754
D:\230217_Lydo_Concerts\CSV\FASTACC-LS410-5022154-23021600-cut.csv
100.14596366882324
D:\230217_Lydo_Concerts\CSV\RESPACC-LS410-5022154-23021600-cut.csv
108.25001168251038
D:\230217_Lydo_Concerts\CSV\RR-LS410-5022154-23021600-cut.csv
108.49950194358826
112.94700741767883
[112.94700741767883, 112.99034786224365]
D:\230217_Lydo_Concerts\CSV\DATA-VN102-3420426-23021700-cut.csv
0.03588104248046875
D:\230217_Lydo_Concerts\CSV\ECG-VN102-3420426-23021700-cut.csv
34.82058238983154
D:\230217_Lydo_Concerts\CSV\FASTACC-VN102-3420426-23021700-cut.csv
60.39468479156494
D:\230217_Lydo_Concerts\CSV\RESPACC-VN102-3420426-23021700-cut.csv
63.72457766532898
D:\230217_Lydo_Concerts\CSV\RR-VN102-3420426-23021700-cut.csv
63.90501308441162
64.65855503082275
[64.6595516204834, 177.64989948272705]
D:\230217_Lydo_Concerts\CSV\DATA-VN205-5022060-230

31.679401874542236
D:\230217_Lydo_Concerts\CSV\FASTACC-VN103-3420427-23021700-cut.csv
56.911027669906616
D:\230217_Lydo_Concerts\CSV\RESPACC-VN103-3420427-23021700-cut.csv
59.9688081741333
D:\230217_Lydo_Concerts\CSV\RR-VN103-3420427-23021700-cut.csv
60.068474769592285
60.824944734573364
[60.824944734573364, 1238.7359342575073]
D:\230217_Lydo_Concerts\CSV\DATA-VN105-3420755-23021700-cut.csv
0.025909900665283203
D:\230217_Lydo_Concerts\CSV\ECG-VN105-3420755-23021700-cut.csv
32.43333864212036
D:\230217_Lydo_Concerts\CSV\FASTACC-VN105-3420755-23021700-cut.csv
59.77073431015015
D:\230217_Lydo_Concerts\CSV\RESPACC-VN105-3420755-23021700-cut.csv
62.92214584350586
D:\230217_Lydo_Concerts\CSV\RR-VN105-3420755-23021700-cut.csv
63.00486516952515
64.05136847496033
[64.05136847496033, 1302.7873027324677]
D:\230217_Lydo_Concerts\CSV\DATA-VN108-5022009-23021700-cut.csv
0.03459477424621582
D:\230217_Lydo_Concerts\CSV\ECG-VN108-5022009-23021700-cut.csv
33.298736572265625
D:\230217_Lydo_Concerts\CSV\FA

55.99099636077881
D:\230217_Lydo_Concerts\CSV\RR-WW507-5022173-23021700-cut.csv
56.09864282608032
56.89108371734619
[56.89108371734619, 2322.017906665802]
D:\230217_Lydo_Concerts\CSV\DATA-WW503-5022161-23021700-cut.csv
0.03289031982421875
D:\230217_Lydo_Concerts\CSV\ECG-WW503-5022161-23021700-cut.csv
29.132910013198853
D:\230217_Lydo_Concerts\CSV\FASTACC-WW503-5022161-23021700-cut.csv
55.77445387840271
D:\230217_Lydo_Concerts\CSV\RESPACC-WW503-5022161-23021700-cut.csv
58.34071683883667
D:\230217_Lydo_Concerts\CSV\RR-WW503-5022161-23021700-cut.csv
58.4722843170166
59.2428514957428
[59.2428514957428, 2381.260758161545]
D:\230217_Lydo_Concerts\CSV\DATA-WW502-5022156-23021700-cut.csv
0.025913000106811523
D:\230217_Lydo_Concerts\CSV\ECG-WW502-5022156-23021700-cut.csv
29.76500415802002
D:\230217_Lydo_Concerts\CSV\FASTACC-WW502-5022156-23021700-cut.csv
54.49737310409546
D:\230217_Lydo_Concerts\CSV\RESPACC-WW502-5022156-23021700-cut.csv
57.4186053276062
D:\230217_Lydo_Concerts\CSV\RR-WW502-502

35.909738302230835
D:\230217_Lydo_Concerts\CSV\FASTACC-PC705-5022245-23021700-cut.csv
58.2323637008667
D:\230217_Lydo_Concerts\CSV\RESPACC-PC705-5022245-23021700-cut.csv
60.19710564613342
D:\230217_Lydo_Concerts\CSV\RR-PC705-5022245-23021700-cut.csv
60.31770610809326
61.1020827293396
[61.1020827293396, 3452.6395976543427]
D:\230217_Lydo_Concerts\CSV\DATA-PC703-5022241-23021700-cut.csv
0.024916887283325195
D:\230217_Lydo_Concerts\CSV\ECG-PC703-5022241-23021700-cut.csv
30.088386058807373
D:\230217_Lydo_Concerts\CSV\FASTACC-PC703-5022241-23021700-cut.csv
54.71183156967163
D:\230217_Lydo_Concerts\CSV\RESPACC-PC703-5022241-23021700-cut.csv
57.66700267791748
D:\230217_Lydo_Concerts\CSV\RR-PC703-5022241-23021700-cut.csv
57.679959297180176
58.47729301452637
[58.47729301452637, 3511.116890668869]
D:\230217_Lydo_Concerts\CSV\DATA-PC701-5022238-23021700-cut.csv
0.025913476943969727
D:\230217_Lydo_Concerts\CSV\ECG-PC701-5022238-23021700-cut.csv
35.44333553314209
D:\230217_Lydo_Concerts\CSV\FASTACC

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,LS402,5022139,230217,0,DATA-LS402-5022139-23021700-cut.csv,csv,64375,2023-02-17 08:15:01+00:00,2023-02-17 11:29:46+00:00,...,[LS402],5022139.0,LS402,NaN,90.0,97.0,0.0,0.0,Stationary,Upright
1,DATA,VA306,5022126,230217,0,DATA-VA306-5022126-23021700-cut.csv,csv,64171,2023-02-17 08:15:02+00:00,2023-02-17 11:29:47+00:00,...,[VA306],5022126.0,VA306,NaN,80.0,98.0,0.0,0.0,Stationary,Upright
2,DATA,LS404,5022142,230217,0,DATA-LS404-5022142-23021700-cut.csv,csv,64188,2023-02-17 08:15:02+00:00,2023-02-17 11:29:47+00:00,...,[LS404],5022142.0,LS404,NaN,78.0,98.0,0.0,0.0,Stationary,Upright
3,DATA,LS407,5022149,230217,0,DATA-LS407-5022149-23021700-cut.csv,csv,64392,2023-02-17 08:15:02+00:00,2023-02-17 11:29:47+00:00,...,[LS407],5022149.0,LS407,NaN,84.0,98.0,0.0,0.0,Stationary,Upright
4,DATA,VA307,5022127,230217,0,DATA-VA307-5022127-23021700-cut.csv,csv,64210,2023-02-17 08:15:02+00:00,2023-02-17 11:29:47+00:00,...,[VA307],5022127.0,VA307,NaN,71.0,98.0,0.0,0.0,Stationary,Upright
5,DATA,VN104,3420456,230217,0,DATA-VN104-3420456-23021700-cut.csv,csv,64880,2023-02-17 08:15:03+00:00,2023-02-17 11:29:48+00:00,...,[VN104],3420456.0,VN104,NaN,83.0,98.0,0.0,0.0,MovingSlowly,Upright
6,DATA,VA303,5022104,230217,0,DATA-VA303-5022104-23021700-cut.csv,csv,63677,2023-02-17 08:15:05+00:00,2023-02-17 11:29:50+00:00,...,[VA303],5022104.0,VA303,NaN,146.0,71.0,0.0,1.0,Stationary,Prone
7,DATA,VA301,5022100,230217,0,DATA-VA301-5022100-23021700-cut.csv,csv,64238,2023-02-17 08:15:05+00:00,2023-02-17 11:29:50+00:00,...,[VA301],5022100.0,VA301,NaN,76.0,98.0,0.0,0.0,Stationary,Upright
8,DATA,PC702,5022240,230217,0,DATA-PC702-5022240-23021700-cut.csv,csv,64217,2023-02-17 08:15:05+00:00,2023-02-17 11:29:50+00:00,...,[PC702],5022240.0,PC702,NaN,68.0,98.0,0.0,0.0,Stationary,Upright
9,DATA,VN101,3420415,230217,0,DATA-VN101-3420415-23021700-cut.csv,csv,65393,2023-02-17 08:15:05+00:00,2023-02-17 11:29:50+00:00,...,[VN101],3420415.0,VN101,NaN,74.0,98.0,0.0,0.0,MovingSlowly,Upright


In [3]:
1758/60

29.3

In [82]:
dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230214_Equivital\
230214_Equivital


,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VA304,5022110,230214,0,DATA-VA304-5022110-23021400-cut.csv,csv,80491,2023-02-14 08:55:02+00:00,2023-02-14 12:59:47+00:00,...,[VA304],5022110.0,VA304,NaN,63,98.0,0.0,0.0,Stationary,Upright
1,DATA,BR604,5022198,230214,101,DATA-BR604-5022198-23021400_0-cut.csv,csv,80215,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[BR604],5022198.0,BR604,NaN,65,98.0,0.0,0.0,Stationary,Upright
2,DATA,VN205,5022060,230214,0,DATA-VN205-5022060-23021400-cut.csv,csv,80246,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[VN205],5022060.0,VN205,NaN,63,99.0,0.0,0.0,Stationary,Upright
3,DATA,PC702,5022240,230214,0,DATA-PC702-5022240-23021400-cut.csv,csv,80308,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[PC702],5022240.0,PC702,NaN,70,98.0,0.0,0.0,Stationary,Upright
4,DATA,VN202,5022051,230214,0,DATA-VN202-5022051-23021400-cut.csv,csv,80336,2023-02-14 08:55:04+00:00,2023-02-14 12:59:51+00:00,...,[VN202],5022051.0,VN202,NaN,84,99.0,0.0,0.0,Stationary,Upright
5,DATA,PC703,5022241,230214,0,DATA-PC703-5022241-23021400-cut.csv,csv,80996,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC703],5022241.0,PC703,NaN,74,100.0,0.0,0.0,Stationary,Upright
6,DATA,LS401,5022138,230214,0,DATA-LS401-5022138-23021400-cut.csv,csv,80302,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[LS401],5022138.0,LS401,NaN,70,99.0,0.0,0.0,Stationary,Upright
7,DATA,PC701,5022238,230214,0,DATA-PC701-5022238-23021400-cut.csv,csv,80391,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC701],5022238.0,PC701,NaN,80,98.0,0.0,0.0,Stationary,Upright
8,DATA,WW502,5022156,230214,0,DATA-WW502-5022156-23021400-cut.csv,csv,80378,2023-02-14 08:55:06+00:00,2023-02-14 12:59:51+00:00,...,[WW502],5022156.0,WW502,NaN,70,98.0,0.0,0.0,Stationary,Upright
9,DATA,LS403,5022141,230214,0,DATA-LS403-5022141-23021400-cut.csv,csv,80401,2023-02-14 08:55:07+00:00,2023-02-14 12:59:52+00:00,...,[LS403],5022141.0,LS403,NaN,88,98.0,0.0,0.0,Stationary,Upright


## Troubleshooting

In [69]:
shutil.move(fi,out_f)

'D:\\230214_Equivital\\SEM\\VN101\\23021400.SEM'

In [11]:
dfiles = qex.qiosk_recordings(discardPath, 'ToBeDiscarded',seperator)
print(len(dfiles))

585
